In [1]:
import sys
import os
sys.path.append('..')


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy import signal
import ipywidgets as widgets
from IPython.display import display

In [3]:
import matplotlib.pyplot as plt
from ipywidgets import *
from src.utils import *
from src.models import *
from src.metrics import *
from modules.degradation import *
from modules.downsample import simulate_lr_from_hr
from modules.upsample import *

In [4]:
hr = load_hr_image("../data/input_hr/brain_ct.png", scale=4)
kernel = gaussian_kernel(9, 1.6)
scale = 4

In [5]:
def restore(lr, scale, model="Nearest", iterations=50, alpha=0.2):
    print(model)
    if model == "Bicubic":
        up = upsample_bicubic
    elif model == "Bilinear":
        up = upsample_bilinear
    elif model == "Nearest":
        up = upsample_nearest
    else:
        raise ValueError("Unknown upsample model")
    return iterative_backprojection(lr, upsample=up, scale=scale, iterations=iterations, alpha=alpha)


In [ ]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

# Noise: chỉ chọn 1
noise_widget = widgets.Dropdown(
    options=["gaussian","rayleigh","gamma","exponential","uniform","saltpepper"],
    value="gaussian",
    description="Noise:"
)

# Models: checkbox cho từng model
model_nearest = widgets.Checkbox(value=True, description="Nearest")
model_bilinear    = widgets.Checkbox(value=False, description="Bilinear")
model_bicubic      = widgets.Checkbox(value=False, description="Bicubic")

model_box = widgets.VBox([model_nearest, model_bilinear, model_bicubic])

# Sliders
iterations_widget = widgets.IntSlider(min=10, max=200, step=10, value=50, description="Iterations")
alpha_widget = widgets.FloatSlider(min=0.0, max=1.0, step=0.05, value=0.2, description="Alpha")

# Button để chạy
run_button = widgets.Button(description="Run SR")

def on_run_clicked(b):
    clear_output(wait=True)
    # Lấy noise
    noise = noise_widget.value

    # Lấy models tick
    models = []
    if model_nearest.value: models.append("Nearest")
    if model_bilinear.value: models.append("Bilinear")
    if model_bicubic.value: models.append("Bicubic")

    iterations = iterations_widget.value
    alpha = alpha_widget.value

    # Tạo LR
    lr = degrade_image(hr, scale=scale, kernel=kernel, noise_type=noise)

    # Hiển thị HR và LR
    n_models = len(models)
    plt.figure(figsize=(4*(n_models+2),4))
    plt.subplot(1,n_models+2,1)
    plt.imshow(hr, cmap="gray"); plt.title("HR gốc"); plt.axis("off")

    plt.subplot(1,n_models+2,2)
    plt.imshow(lr, cmap="gray"); plt.title(f"LR ({noise})"); plt.axis("off")

    # SR cho từng model đã tick
    for i, m in enumerate(models):
        # if m == "IBP-Bicubic":
        #     sr = iterative_backprojection(lr, scale, kernel, iterations, alpha)
        # else:
        #     continue
        sr = restore(lr, scale, m, iterations, alpha)

        plt.subplot(1,n_models+2,i+3)
        plt.imshow(sr, cmap="gray"); plt.title(f"SR ({m})"); plt.axis("off")
        print(f"PSNR ({m} vs HR):", psnr(sr, hr))

    plt.show()

run_button.on_click(on_run_clicked)

# Hiển thị tất cả widget
display(noise_widget, model_box, iterations_widget, alpha_widget, run_button)

Dropdown(description='Noise:', options=('gaussian', 'rayleigh', 'gamma', 'exponential', 'uniform', 'saltpepper…

IntSlider(value=50, description='Iterations', max=200, min=10, step=10)

FloatSlider(value=0.2, description='Alpha', max=1.0, step=0.05)

Button(description='Run SR', style=ButtonStyle())